# Studi Kasus:
**Kita akan mengambil informasi pembayaran dari tabel payment dan menggabungkannya dengan informasi pelanggan dari tabel customer, serta menambahkan informasi alamat lengkap dari tabel address. Setelah itu, kita akan melakukan konversi format tanggal pembayaran menjadi format yang lebih mudah dipahami.**

In [19]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

# Fungsi untuk membersihkan data pada kolom numerik dan kategorikal
def clean_data(df):
    # Menghitung proporsi missing value pada setiap kolom
    missing_prop = df.isnull().mean()

    # Menghapus baris jika proporsi missing value dibawah 10%
    rows_to_drop = missing_prop[missing_prop < 0.10].index.tolist()
    df = df.dropna(subset=rows_to_drop)

    # Kolom-kolom numerik: Mengisi nilai null dengan mean jika terdistribusi normal, dan median jika tidak
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            if df[col].dtype != 'object':
                if df[col].skew() > 1 or df[col].skew() < -1:  # Jika distribusi tidak normal (skewness > 1 atau < -1)
                    df[col] = df[col].fillna(df[col].median())
                else:  # Jika terdistribusi normal
                    df[col] = df[col].fillna(df[col].mean())

    # Kolom-kolom kategorikal: Mengisi nilai null dengan mode
    categorical_cols = df.select_dtypes(include='object').columns.tolist()
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].mode()[0])

    return df

# Fungsi ETL dengan dataset Sakila dan data cleaning menggunakan metode statistik
def run_etl_with_sakila_data():
    # Mendapatkan kredensial MySQL dari variabel lingkungan (env)
    db_user = os.environ.get('MYSQL_USER')
    db_password = os.environ.get('MYSQL_PASSWORD')
    db_host = 'localhost'
    db_name = 'sakila'

    # Koneksi ke database Sakila di MySQL
    db_connection_str = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'
    db_connection = create_engine(db_connection_str)

    # Baca tabel-tabel yang diperlukan dari database Sakila dengan alamat lengkap
    query = """
        SELECT p.payment_id, c.customer_id AS user_id, CONCAT(c.first_name, ' ', c.last_name) AS full_name,
        a.address AS full_address, a.district, a.postal_code, p.payment_date, f.title AS film_title
        FROM payment p
        INNER JOIN customer c ON p.customer_id = c.customer_id
        INNER JOIN address a ON c.address_id = a.address_id
        INNER JOIN rental r ON p.rental_id = r.rental_id
        INNER JOIN inventory i ON r.inventory_id = i.inventory_id
        INNER JOIN film f ON i.film_id = f.film_id
    """
    df = pd.read_sql(query, con=db_connection)

    # Memanggil fungsi clean_data untuk membersihkan data pada setiap kolom
    df = clean_data(df)

    # Tampilkan informasi pembayaran pelanggan dengan detail film yang dibayar
    print(df.head())

    return df

In [20]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import io

# Fungsi untuk autentikasi ke Google Drive dan unggah data dari pandas DataFrame
def upload_dataframe_to_google_drive(df):
    scope = ['https://www.googleapis.com/auth/drive']

    credentials = ServiceAccountCredentials.from_json_keyfile_name('your_credentials.json', scope)
    gc = gspread.authorize(credentials)

    sh = gc.create('Sakila Payment, Customer & Address Data')
    sh.share('example@example.com', perm_type='user', role='writer')

    # Convert pandas DataFrame ke format CSV
    csv_data = df.to_csv(index=False)

    # Buat StringIO untuk menyimpan data CSV
    csv_io = io.StringIO(csv_data)

    # Unggah data langsung ke Google Drive
    gc.import_csv(sh.id, data=csv_io.read())


In [21]:
# Jalankan proses ETL dengan data cleaning pada dataset Sakila untuk informasi pembayaran
df = run_etl_with_sakila_data()


# Simpan data ke file CSV baru setelah cleaning
new_file_name = 'sakila_payment_details_cleaned.csv'
df.to_csv(new_file_name, index=False)

# Unggah data dari DataFrame yang telah diolah ke Google Drive
# upload_dataframe_to_google_drive(df)

   payment_id  user_id        full_name                  full_address  \
0       11630      431   JOEL FRANCISCO         287 Cuautla Boulevard   
1       13956      518   GABRIEL HARDER  680 A Corua (La Corua) Manor   
2        7578      279   DIANNE SHELTON           600 Bradford Street   
3       11124      411   NORMAN CURRIER           1445 Carmen Parkway   
4        4607      170  BEATRICE ARNOLD               1877 Ezhou Lane   

           district postal_code        payment_date        film_title  
0        Chuquisaca       72736 2005-07-08 19:03:15  ACADEMY DINOSAUR  
1             Sivas       49806 2005-08-02 20:13:10  ACADEMY DINOSAUR  
2  East Azerbaidzan       96204 2005-08-21 21:27:43  ACADEMY DINOSAUR  
3         West Java       70809 2005-05-30 20:21:07  ACADEMY DINOSAUR  
4         Rajasthan       63337 2005-06-17 20:24:00  ACADEMY DINOSAUR  


In [22]:
df.head()

,payment_id,user_id,full_name,full_address,district,postal_code,payment_date,film_title
0,11630,431,JOEL FRANCISCO,287 Cuautla Boulevard,Chuquisaca,72736,2005-07-08 19:03:15,ACADEMY DINOSAUR
1,13956,518,GABRIEL HARDER,680 A Corua (La Corua) Manor,Sivas,49806,2005-08-02 20:13:10,ACADEMY DINOSAUR
2,7578,279,DIANNE SHELTON,600 Bradford Street,East Azerbaidzan,96204,2005-08-21 21:27:43,ACADEMY DINOSAUR
3,11124,411,NORMAN CURRIER,1445 Carmen Parkway,West Java,70809,2005-05-30 20:21:07,ACADEMY DINOSAUR
4,4607,170,BEATRICE ARNOLD,1877 Ezhou Lane,Rajasthan,63337,2005-06-17 20:24:00,ACADEMY DINOSAUR
